In [2]:
import pandas as pd
df_collab = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\pct_collaborative_pubs_top42.csv', index_col = 'name', encoding = "UTF-8")
df_collab_new = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\pct_collaborative_pubs_top42_v2.csv', index_col = 'name', encoding = "UTF-8")
df_disrupt = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\disrupt_profs_top42.csv', index_col = 'name', encoding = "UTF-8")
df_impact = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\.csv', index_col = 'name', encoding = "UTF-8")
df_clusters = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\clusters_top42.csv', index_col = 'name', encoding = "UTF-8")
df_unique_profs = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\unique_profs_top42.csv', index_col = 'name', encoding = "UTF-8")
df_orgs = pd.read_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\orgs_top42.csv', index_col = 'name', encoding = "UTF-8")

FileNotFoundError: [Errno 2] File b'C:\\Users\\Brend\\Downloads\\Collaboration\\pct_collaborative_pubs_top42.csv' does not exist: b'C:\\Users\\Brend\\Downloads\\Collaboration\\pct_collaborative_pubs_top42.csv'

In [3]:
all_dfs = [df_collab_new, df_disrupt, df_impact, df_clusters, df_unique_profs, df_orgs]
df = all_dfs[0]
for i in range(1, len(all_dfs)):
    df = df.join(all_dfs[i], how = 'outer')

In [1]:
print("Total: %d authors" % df.shape[0])
df_collab_trim = df[df['num_pubs'] >= 50]
df_collab_trim = df_collab_trim[df_collab_trim['TotalConns'] >= 25]
df_collab_trim = df_collab_trim[df_collab_trim['orgs'] <= 1000]
print("Remaining after trimming: %d authors" % df_collab_trim.shape[0])

NameError: name 'df' is not defined

In [5]:
df_impact_trim = df_collab_trim

In [6]:
df_disrupt_trim = df[df['num_disrupt'] >= 10]
df_disrupt_trim = df_disrupt_trim[df_disrupt_trim['orgs'] <= 2000]
print("Remaining after trimming: %d authors" % df_disrupt_trim.shape[0])

Remaining after trimming: 57551 authors


In [7]:
df_unique_profs_trim = df_collab_trim

In [10]:
def top_100_to_csv(trimmed_df, cols, name, num = 200, to_csv = False):
    d = dict()
    for col in cols:
        df_notna = trimmed_df[pd.notna(trimmed_df[col])]
        sorted = df_notna[[col]].sort_values(col, ascending = False).head(num)
        names = sorted.index.tolist()
        values = sorted[col].tolist()
        d[col] = pd.Series(names, index=list(range(1, num+1)))
        d[col+'_val'] = pd.Series(values, list(range(1, num+1)))
    df_rank = pd.DataFrame(d)
    if to_csv:
        df_rank.to_csv('C:\\Users\\Brend\\Downloads\\Collaboration\\{}_rank_{}.csv'.format(name, num), index = False, encoding = "UTF-8")
    return df_rank

In [17]:
top_100_to_csv(df_impact_trim, df_impact.columns, 'impact', 1000, to_csv = True)

,avg_PR,avg_PR_val,max_PR,max_PR_val,median_PR,median_PR_val,min_PR,min_PR_val,avg_AR,avg_AR_val,max_AR,max_AR_val,median_AR,median_AR_val,min_AR,min_AR_val
1,Ulrich K. Laemmli,35.501431,Joseph Sambrook,4942.389210,Thomas R. Dawber,2.912980,McBride Ow,0.174004,Ulrich K. Laemmli,18.632654,Oliver H. Lowry,2160.619064,C Redmond,1.185122,McBride Ow,0.165453
2,Stephen F. Altschul,23.041292,Tom Maniatis,4942.389210,Tavia Gordon,2.518032,Dennis A. Senne,0.163371,Stephen F. Altschul,12.874065,Joseph Sambrook,1502.146478,W. M. Cowan,1.163485,Dennis A. Senne,0.159631
3,David J. Lipman,22.622525,Oliver H. Lowry,4713.407514,C Redmond,2.283521,Victor E. Koteliansky,0.159826,David J. Lipman,12.561464,Tom Maniatis,1502.146478,Robert T. DeBoy,1.158474,Victor E. Koteliansky,0.158696
4,Tom Maniatis,20.387078,Ulrich K. Laemmli,2664.815842,Parkhurst A. Shore,2.244719,Colm O'hUigin,0.159444,Oliver H. Lowry,8.934023,Ulrich K. Laemmli,1408.893785,Robert J. Dodson,1.119536,Colm O'hUigin,0.158543
5,Oliver H. Lowry,19.520199,Lars Hedin,2039.665534,W. M. Cowan,1.989251,J J Mekalanos,0.159418,Tom Maniatis,7.046123,Paul Meier,600.372834,Hoda Khouri,1.116985,J J Mekalanos,0.157327
6,Joseph Sambrook,17.173986,Paul Meier,1335.049007,Arthur Purdy Stout,1.968363,K. H. Cowan,0.156571,Mark S. Guyer,5.915697,Nicoletta Sacchi,422.102206,Tavia Gordon,1.105510,K. H. Cowan,0.154055
7,Lars Hedin,14.926409,Linus Pauling,1062.882866,G. E. Palade,1.736083,B. A. Roe,0.154786,Kary B. Mullis,5.502184,Marshal F. Folstein,383.737923,Thomas R. Dawber,1.097980,B. A. Roe,0.153842
8,Kary B. Mullis,13.438516,David S. Johnson,1062.522355,A. L. Hodgkin,1.669588,Donna Van Engen,0.154139,Theophil Staehelin,5.422972,Susan E. Folstein,383.737923,Ellen E. Walters,1.015886,Jun Lin Guan,0.153485
9,William H. Stein,12.567903,Max Born,1033.047322,J L Goldstein,1.615923,Per Olof Nyman,0.154046,Joseph Sambrook,5.399066,Saul A. Teukolsky,379.590731,W. James Kent,0.985605,Per Olof Nyman,0.153383
10,Randall K. Saiki,12.030608,E. Wolf,1033.047322,John C. Cassel,1.557405,Jun Lin Guan,0.153697,Koichiro Tamura,5.380320,Eugene W. Myers,377.100282,J L Goldstein,0.966387,Jana S. McBride,0.152729


In [13]:
top_100_to_csv(df_disrupt_trim, df_disrupt.columns, 'disrupt', to_csv = True)

,avg_disrupt,avg_disrupt_val,max_disrupt,max_disrupt_val,median_disrupt,median_disrupt_val,min_disrupt,min_disrupt_val,num_disrupt,num_disrupt_val
1,Gary Taubes,0.341513,Liang Tong,0.998155,Gary Taubes,0.280335,D. Rittenberg,0.026923,Philip Ball,723.0
2,D. Rittenberg,0.332206,Derek Yach,0.996011,N. Xeros,0.275862,Victor L. Loosanoff,0.023622,Richard C. Cabot,510.0
3,Emily Singer,0.316465,Sidney Katz,0.995929,Emily Singer,0.250000,Donald J. Cram,0.008400,Ben Short,483.0
4,R. A. Shooter,0.305285,A. Binz,0.994819,L Roberts,0.214286,Dorothy Crowfoot Hodgkin,0.006822,Richard A. Flavell,429.0
5,Alfred A. Strauss,0.305140,David P. Fairlie,0.993939,D. Rittenberg,0.213065,John H. Fisher,0.006527,Mitch Leslie,410.0
6,Wylie McKissock,0.295792,Jennifer L. Martin,0.993939,Howard W. Haggard,0.207885,G. Davidson,0.006250,Carlo M. Croce,362.0
7,N. Xeros,0.286080,James R. Heath,0.993902,Alfred A. Strauss,0.201923,E. M. Widdowson,0.004785,Nancy Lee Harris,355.0
8,L Roberts,0.278846,Steven C. Clark,0.993506,R. A. Shooter,0.200612,Emily Friedman,0.004587,Tak W. Mak,350.0
9,Herbert C. Brown,0.268188,Giorgio Trinchieri,0.993506,Mark Crawford,0.200000,Takesi Nagata,0.004255,Eugene J. Mark,344.0
10,Elliott P. Joslin,0.263483,Michiko Kobayashi,0.993506,C. G. Overberger,0.193350,Pedro A. Sanchez,0.003992,Richard Horton,331.0


In [15]:
top_100_to_csv(df_unique_profs_trim, df_unique_profs.columns, 'unique_profs', to_csv = True)

,unique_coauthor_profs,unique_coauthor_profs_val,unique_coauthor_profs_exp_damp,unique_coauthor_profs_exp_damp_val,unique_coauthor_profs_lin_damp,unique_coauthor_profs_lin_damp_val,unique_coauthor_profs_sqrt_damp,unique_coauthor_profs_sqrt_damp_val,unique_coauthor_profs_square_damp,unique_coauthor_profs_square_damp_val
1,Albert Hofman,5561.0,Barry M. Trost,28.928570,Hagop Kantarjian,886.398632,Eric S. Lander,1667.474527,Richard A. Flavell,483.375030
2,Kari Stefansson,5134.0,Annalisa M. VanHook,25.414127,Shizuo Akira,841.319270,Kari Stefansson,1490.158500,Shizuo Akira,455.462193
3,André G. Uitterlinden,5076.0,E. J. Corey,20.855733,Kenneth C. Anderson,809.261427,Albert Hofman,1489.394965,Hagop Kantarjian,437.892975
4,D. H. Miller,4739.0,Eugene J. Mark,18.643059,Richard A. Flavell,806.393284,Richard K. Wilson,1436.096992,Carlo M. Croce,427.403403
5,T. Williams,4601.0,Roger Adams,16.286741,Gordon B. Mills,768.034885,Richard A. Gibbs,1431.898313,Tak W. Mak,401.883865
6,Eric S. Lander,4579.0,Benjamin Castleman,15.352905,Eric S. Lander,765.986446,Mark J. Daly,1411.844582,Kenneth C. Anderson,396.030779
7,A. Castro,4452.0,Herbert C. Brown,15.250789,Carlo M. Croce,757.826888,André G. Uitterlinden,1407.663097,al. et,369.415448
8,Mark J. Daly,4452.0,John D. Roberts,13.242179,Tak W. Mak,705.793004,Stacey Gabriel,1383.036115,Gordon B. Mills,369.385937
9,J. Haas,4437.0,Leo A. Paquette,12.536955,Abstr Act,690.260104,Stephen J. Chanock,1285.454075,Roderick T. Bronson,367.005759
10,A. Santoro,4417.0,Thomas C. Bruice,12.492976,Yusuke Nakamura,673.814813,Hagop Kantarjian,1284.775482,John R. Yates,350.378010


In [14]:
interesting_authors = ['Eric S. Lander', 'Robert Langer', 'Francis Crick', 'Sydney Brenner', 'Richard Henderson', 'Edward S. Boyden']
def find_percentiles(trimned_df, cols, authors = interesting_authors):
    df_percentile = trimned_df.copy()
    for col in cols:
        percentile_col = trimned_df[pd.notna(trimned_df[col])][col]
        df_percentile[col+'_%'] = percentile_col.rank(pct=True)
    df_percentile_authors = df_percentile.loc[authors][[col+'_%' for col in cols]]
    return df_percentile_authors

In [22]:
find_percentiles(df_impact_trim, df_impact.columns)

,avg_PR_%,max_PR_%,median_PR_%,min_PR_%,avg_AR_%,max_AR_%,median_AR_%,min_AR_%
name,,,,,,,,
Eric S. Lander,0.999335,0.998525,0.991882,0.499976,0.999555,0.998884,0.993229,0.499976
Robert Langer,0.942649,0.984560,0.698009,0.499976,0.947837,0.988143,0.688794,0.499976
Francis Crick,0.995950,0.997445,0.090087,0.499976,0.990795,0.995088,0.090087,0.499976
Sydney Brenner,0.996532,0.999408,0.792797,0.499976,0.993207,0.998107,0.770642,0.499976
Richard Henderson,0.852040,0.760284,0.471854,0.499976,0.840116,0.771833,0.410736,0.499976
Edward S. Boyden,0.444957,0.886514,0.090087,0.499976,0.492899,0.912890,0.090087,0.499976


In [23]:
find_percentiles(df_disrupt_trim, df_disrupt.columns)

,avg_disrupt_%,max_disrupt_%,median_disrupt_%,min_disrupt_%,num_disrupt_%
name,,,,,
Eric S. Lander,0.613543,0.809743,0.277450,0.155393,0.999739
Robert Langer,0.803114,0.897717,0.818865,0.189128,0.998436
Francis Crick,0.982572,0.996707,0.983632,0.035186,0.819343
Sydney Brenner,0.864103,0.868047,0.858421,0.221299,0.950305
Richard Henderson,0.492554,0.633151,0.370524,0.443919,0.281620
Edward S. Boyden,0.831124,0.963650,0.796841,0.179554,0.925527


In [15]:
find_percentiles(df_unique_profs_trim, df_unique_profs.columns)

,unique_coauthor_profs_%,unique_coauthor_profs_exp_damp_%,unique_coauthor_profs_lin_damp_%,unique_coauthor_profs_sqrt_damp_%,unique_coauthor_profs_square_damp_%
name,,,,,
Eric S. Lander,0.999986,0.983350,0.999986,1.000000,0.999915
Robert Langer,0.980867,0.998970,0.999739,0.999030,0.999827
Francis Crick,0.670713,0.977414,0.692417,0.662714,0.735177
Sydney Brenner,0.956361,0.995599,0.986060,0.977309,0.988755
Richard Henderson,0.677760,0.937031,0.724602,0.686574,0.753870
Edward S. Boyden,0.939014,0.951800,0.985833,0.968240,0.988137
